### 3. Classifiers

This notebook to classify whether fake news will travel from A to B.

In [1]:
%pylab inline
import pandas as pd, pyprind

from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score,make_scorer

Populating the interactive namespace from numpy and matplotlib


In [2]:
ls data

ClassificationModelInput.csv  pol_agg.csv
emergent.csv*                 politifact.csv*
fake_localcentralities.csv    politifact_clean.csv
key_mutuality_roshan.csv      real_localcentralities.csv
keys.csv                      snopes.csv*


In [3]:
d=pd.read_csv('data/ClassificationModelInput.csv')
sum(d.isnull()) #No NA

Source                                      0
Destination                                 0
Total Volume                                0
TRUE                                        0
FAKE                                        0
Source_Real_LocalDegreeCentralities         0
Destination_Real_LocalDegreeCentralities    0
Source_Real_LocalBetweenness                0
Destination_Real_LocalBetweenness           0
Source_Real_LocalCloseness                  0
Destination_Real_LocalCloseness             0
Source_Real_LocalEigenCentralities          0
Destination_Real_LocalEigenCentralities     0
Source_Fake_LocalDegreeCentralities         0
Destination_Fake_LocalDegreeCentralities    0
Source_Fake_LocalBetweenness                0
Destination_Fake_LocalBetweenness           0
Source_Fake_LocalCloseness                  0
Destination_Fake_LocalCloseness             0
Source_Fake_LocalEigenCentralities          0
Destination_Fake_LocalEigenCentralities     0
jaccard_coeff_connection          

In [4]:
d['Label']=d['FAKE'].apply(lambda x: int(x>0))

#Check for class imblanace
print(sum(d['Label']==0)/len(d))

#Ready for model
X=d[['Source_Fake_LocalDegreeCentralities',
       'Destination_Fake_LocalDegreeCentralities',
       'Source_Fake_LocalBetweenness', 'Destination_Fake_LocalBetweenness',
       'Source_Fake_LocalCloseness', 'Destination_Fake_LocalCloseness',
       'Source_Fake_LocalEigenCentralities',
       'Destination_Fake_LocalEigenCentralities', 'jaccard_coeff_connection',
       'Neighbor_connection', 'jaccard_coeff_common_destination',
       'Neighbor_common_destination', 'jaccard_coeff_common_source',
       'Neighbor_common_source', 'mutuality_ind']]
Y=d['Label']

0.559619604974


In [35]:
#Random Forest, GBM

#Make AUC the performance metric
auc_scorer=make_scorer(roc_auc_score,
                       greater_is_better=True)

models = {'RF':{'model':RandomForestClassifier(),
                     'Params':{'n_estimators':range(12,20),
                              'max_depth':range(2,5),
                              'min_samples_leaf':range(3,8)}},
         'GBM':{'model':GradientBoostingClassifier(),
                     'Params':{'n_estimators':range(40,100,20),
                               'max_depth':range(3,10),
                               'learning_rate':linspace(0.1,0.8,num=5),
                               'min_samples_leaf':range(2,8)}}}

In [36]:
print('Number of features:', len(X.columns))
print('Number of records', len(d))

Number of features: 15
Number of records 2734


In [37]:
def modeleva(cvfolds):
    allmodels={}
    for model in models.keys():
        grid = GridSearchCV(estimator=models[model]['model'], 
                            param_grid=models[model]['Params'],
                            cv=cvfolds, n_jobs=-1,scoring=auc_scorer)
        modelstats={}
        modelstats['model']=grid.fit(X, Y)
        modelstats['best_estimator']=modelstats['model'].best_estimator_
        modelstats['best_score']=modelstats['model'].best_score_
        allmodels[model]=modelstats
        del modelstats
        print(model)
    results=pd.DataFrame.from_dict(allmodels).T.sort_values(by='best_score')
    results=results.reset_index()
    return(results)

In [39]:
replicates=3
allresults=[modeleva(5) for i in range(replicates)]
allresults=pd.concat(allresults)

RF
GBM
RF
GBM
RF
GBM


In [40]:
print((allresults.groupby('index')['best_score'].sum()/replicates).sort_values(ascending=False))
bestmodel=(allresults.groupby('index')['best_score'].sum()/replicates).sort_values(ascending=False).index[0]
bestmodel=allresults[allresults['index']==bestmodel]['best_estimator'].tolist()[0]

index
GBM    0.580503
RF     0.572392
Name: best_score, dtype: float64


In [41]:
bestmodel

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.10000000000000001, loss='deviance',
              max_depth=4, max_features=None, max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=2,
              min_samples_split=2, min_weight_fraction_leaf=0.0,
              n_estimators=40, presort='auto', random_state=None,
              subsample=1.0, verbose=0, warm_start=False)